In [2]:
#1
import numpy as np
%pylab 
style.use('ggplot')
rcParams['figure.figsize'] = (7,7)
from numba import jit

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import copy
import networkx as nx

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
def getRandomConnectivity(N, pct_connected):
    #Directed mapping
    M = np.random.rand(N**2).reshape(N,N)
    for i in range(N):
        for j in range(N):
            if (i == j):
                M[i,j] = 0
            else:
                if (M[i,j] < pct_connected):
                    M[i,j] = 1 #i connects to j
                else:
                    M[i,j] = 0
    return M

# alternate version
def getRandomConnectivity2(N, pct_connected):
    # There are N*(N-1)/2 upper diagonal elements
    rand_elems = np.where(np.random.rand(N*(N-1)/2) < pct_connected, 1, 0)
    M = np.zeros((N,N))
    M[np.triu_indices(N,1)] = rand_elems
    return M + M.T

# Original version with bidirectional mapping 
def getRandomConnectivity3(N, pct_connected):
    # There is no self coupling so the diagonal must be zero. 
    # The graph is non directed so M must be symmetric
    M = np.random.rand(N**2).reshape(N,N)
    for i in range(N):
        for j in range(i, N):
            if (i == j):
                M[i,j] = 0
            else:
                if (M[i,j] < pct_connected):
                    M[i,j] = 1
                else:
                    M[i,j] = 0
                M[j,i] = M[i,j]
    return M

In [3]:
def hasKcore(M, k):
    # we'll be destroying the array so make a copy to work with
    X = M.copy()
    while(True):
        cur_num_nodes = X.shape[0]
        s = np.sum(X, 0)
        nodes_to_delete = np.where(s < k)[0]
        if (len(nodes_to_delete) == cur_num_nodes):
            # nothing has at least k connections
            
            X = np.delete(X, nodes_to_delete, axis=0)
            X = np.delete(X, nodes_to_delete, axis=1)
            
            return False
            break
        elif (len(nodes_to_delete) == 0):
            # They all have at least k connections, we've found a kcore
            return True
            break
        else:
            X = np.delete(X, nodes_to_delete, axis=0)
            X = np.delete(X, nodes_to_delete, axis=1)

            
# TODO get Kcore indices 

def largestKcore(M):
    # we can begin the search at the least connected node
    connectivity = np.sum(M,0)
    min_kcore = np.min(connectivity)
    max_kcore = np.max(connectivity)
    if max_kcore == 0:
        return 0
    k = min_kcore
    while (hasKcore(M,k)):
        k = k+1
        
    return k - 1
        
    
    
def largestkcore(M):
    # we can begin the search at the least connected node
    connectivity = np.sum(M,0)
    min_kcore = np.min(connectivity)
    max_kcore = np.max(connectivity)
    if max_kcore == 0:
        return 0
    k = min_kcore
    s=1
    while (s==1):
        
        X = M.copy()
        while(True):
            cur_num_nodes = X.shape[0]
            s = np.sum(X, 0)
            nodes_to_delete = np.where(s < k)[0]
            if (len(nodes_to_delete) == cur_num_nodes):
            # nothing has at least k connections
            
                X = np.delete(X, nodes_to_delete, axis=0)
                X = np.delete(X, nodes_to_delete, axis=1)
            
                s=0
                break
            elif (len(nodes_to_delete) == 0):
            # They all have at least k connections, we've found a kcore
                s=1
                break
            else:
                X = np.delete(X, nodes_to_delete, axis=0)
                X = np.delete(X, nodes_to_delete, axis=1)
        k = k+1
        
    return k - 1

#TODO: Instead of deleting nodes, put 0's on all the connections for the rows and columns that are disconnected
    #Keeps number of all neurons the same so that neuron 3 doesnt become neuron 2 etc. 
    #Look for neurons not in Kcore which are firing and neurons that are in kcore and not firing. 
def kcoreitself(M, k):
    # we'll be destroying the array so make a copy to work with
    X = M.copy()
    
    while(True):
        cur_num_nodes = X.shape[0]
        s = np.sum(X, 0)
        nodes_to_delete = np.where(s < k)[0]
        if (len(nodes_to_delete) == cur_num_nodes):
            
            X = np.delete(X, nodes_to_delete, axis=0)
            X = np.delete(X, nodes_to_delete, axis=1)
            # nothing has at least k connections
            return X
            break
        elif (len(nodes_to_delete) == 0):
            # They all have at least k connections, we've found a kcore
            return X
            break
        else:
            X = np.delete(X, nodes_to_delete, axis=0)
            X = np.delete(X, nodes_to_delete, axis=1)



#here we get the largest connected component of  the graph

def largestcomponent(l,N):
    n=np.random.randint(0,N)
    
    boundary=[n]
    b=1
    comp=[n]
    c=1
   
    while(b>0):
        a=boundary.pop()
        
        oldvertex=int(a)
        
        deg=l[oldvertex,0]
        degree=int(deg)
        #print(degree)
        for j in range(degree):
            newvertex=l[oldvertex,j+1]
            q=comp.count(newvertex)
            #print(newvertex)
            q=int(q)
            #print(q)
            if (q==0):
                comp.append(newvertex)
                boundary.append(newvertex)
                #print(newvertex)
                #print(boundary)
            
        b=len(boundary)
            
    #sizeofcomp=len(comp)
    return(comp)

def matrixOfEdges(M,N):
    N = int(N)
    E=np.zeros(N**2).reshape(N,N)  #prepare array with zeros
    k=np.sum(M,0) #array with degree of each vertex
    for i in range(N):
        a=int(k[i]) #degree of the current vertex
        E[i,0]=a  #we put it to the zero row of matrix of edges
    for i in range(N):
        a=int(k[i])
        q=1
        for j in range(N):
            if (M[j,i]==1):
                E[i,q]=j #all the next elements in current column are number of vertices current vertex is connected to
                q=q+1
    return(E)

In [4]:
def simdyn(M,l,params):
    return simDynamics(M,l,params)

@jit(nopython=True)
def simDynamics(M,l,params):
    N=int(params[0])
    n=int(params[1])
    dt=params[2]
    tauv=params[3]
    tauc=params[4]
    Vstar=params[5]
    Cstar=params[6]
    delV=params[7]
    delC=params[8]
    r=params[9]
    V0=params[10]
    C0=params[11]
    V=zeros(N*n).reshape(N,n)  #now we have potencial and calcium concentration for each neuron
    C=zeros(N*n).reshape(N,n)
    P=zeros(n)  # function P(V), sigmoid
    sump=zeros(n)  #sum of P over the vertices
    avV=zeros(N) #average potencial
    avC=zeros(N)  #average calcium
    #res = int(params[12]) #resulting classification 
    firing_neurons_count = zeros(N*n).reshape(N,n) #Individually count how may neurons are actually firing in this model at a given step N.
    
    for i in range(n):
        V[0,i]=V0 #initial conditions
        C[0,i]=C0 
        if V[0,i] > Vstar:
            firing_neurons_count[0,i] = 1
        avV[0]=n*V0  #we will divide to n later
        avC[0]=n*C0
    for i in range (N-1):
        for j in range(n):
            P[j] = 1/(1+exp((Vstar-V[i,j])*0.2))
        for j in range(n):
            a=int(l[j,0])
            for k in range(a):
                b=int(l[j,k])
                sump[j]=sump[j]+P[b]  #if neuron connected to j-th neuron it participates in sum
        for j in range(n):
            V[i+1,j]=(-V[i,j]/tauv + delV*r*sump[j]*(1/(1+exp(C[i,j]-Cstar))) )*dt + V[i,j]
            if V[i+1,j] > Vstar: 
                firing_neurons_count[i+1,j] = 1 #used to return count of firing neurons later
            C[i+1,j]=(-C[i,j]/tauc + delC*r*sump[j])*dt + C[i,j]
            avC[i+1]=avC[i+1]+C[i+1,j]
            avV[i+1]=avV[i+1]+V[i+1,j]
        for j in range(n):
            sump[j]=0
    for i in range (N):
        avC[i]=avC[i]/n
        avV[i]=avV[i]/n
        
   
    return(avC,avV,firing_neurons_count)

In [5]:
import pdb
#for some reason we need this funky embedding
def simdyn3(M, l, params):
    return simDynamics3(M, l, params)



@jit(nopython=True)
def simDynamics3(M, l, params):  #params is a vector of parameters
    N=int(params[0])
    n=int(params[1])
    dt=params[2]
    tauv=params[3]
    tauc=params[4]
    Vstar=params[5]
    Cstar=params[6]
    delV=params[7]
    delC=params[8]
    r=params[9]
    V0=params[10]
    C0=params[11]
    delVmin = params[12]
    V=zeros(N*n).reshape(N,n)  #now we have potencial and calcium concentration for each neuron
    C=zeros(N*n).reshape(N,n)
    #P=zeros(n)  # function P(V), sigmoid
    P = zeros(N*n).reshape(N,n)
    deltaV=zeros(n)
    sump=zeros(n)  #sum of P over the vertices
    avV=zeros(N) #average potencial
    avC=zeros(N)  #average calcium
    firing_neurons_count = zeros(N*n).reshape(N,n) #Individually count how may neurons are actually firing in this model at a given step N.
    
    for i in range(n):
        V[0,i]=V0 #initial conditions
        C[0,i]=C0 
        if V[0,i] > Vstar: #keep track of and return how many neurons in the network are actually firing  
            firing_neurons_count[0,i] = 1
        avV[0]=n*V0  #we will divide to n later
        avC[0]=n*C0
    for i in range (N-1):
        for j in range(n):
            if (V[i,j]>Vstar): #V[i,j] stores voltage of each neuron instead of returning the average voltage 
                P[i,j]=70
            else:
                P[i,j]=5
            if (C[i,j]<Cstar):
                deltaV[j]=delV #change from 1.2 
            else:
                deltaV[j]=delVmin #changed from .4
        for j in range(n):
            a=int(l[j,0]) #a should give number of firing neighbors 
            for k in range(a):
                #b=int(l[j,k])
                b = int(l[j,k+1])
                sump[j]=sump[j]+P[i,b]  #if neuron connected to j-th neuron it participates in sum
        for j in range(n):
            V[i+1,j]=(-V[i,j]/tauv + deltaV[j]*sump[j])*dt + V[i,j]
            if V[i+1,j] > Vstar: 
                firing_neurons_count[i+1,j] = 1 #used to return count of firing neurons later
            #if ((firing_neurons_count[i,j] == 1) and (firing_neurons_count[i+1,j] == 0)):
            #    print(sump[j])
            C[i+1,j]=(-C[i,j]/tauc + delC*r*sump[j])*dt + C[i,j]
            avC[i+1]=avC[i+1]+C[i+1,j]
            avV[i+1]=avV[i+1]+V[i+1,j]
        for j in range(n):
            sump[j]=0
    for i in range (N):
        avC[i]=avC[i]/n
        avV[i]=avV[i]/n
        
        #plot([avC[i],avC[i+1]],[avV[i],avV[i+1]])    
    return(avC,avV, firing_neurons_count, P, V)

In [6]:
def classify(V, params):
    #Set up some globals 
    
    N = int(params[0])
    n = int(params[1])
    
    #Network uniformly connected but vectorized lets us scale later 
    #for i in range (N/2, N):
    #max_V = max(V[int(N/2):N])
    #min_V = min(V[int(N/2):N])
    max_V = max(V[int(.9*N):N])
    min_V = min(V[int(.9*N):N])
   
            
    V_delta = max_V - min_V
    #print(V_delta)
    
    if V_delta < 0.01: #changed from 1
        if V[N-1] < 15:
            #params[12] = 2
            return 2
        else:
            #params[12] = 3
            return 3
    else:
        #params[12] = 1
        return 1

In [7]:
def killNeuron(N,M_prime):
    M = copy.deepcopy(M_prime)
    #Randomly pick a neuron to remove
    #x = np.random.randint(N)
    x = 1
    M = np.delete(M,x,axis=0)
    M = np.delete(M,x,axis=1)
    return M

#TODO: Kill N neurons from the network systematically instead of one at a time. 
def killNeuron2(to_cull, M_prime):
    M = copy.deepcopy(M_prime) 
    #Systemically cull certain neuron from the matrix
    x = 0    
    for i in range (int(to_cull)):
        M = np.delete(M,x,axis=0)
        M = np.delete(M,x,axis=1)
    return M

In [61]:
#Phase Diagrams 
N=10000
n=150 #100
dt=0.001
T=zeros(N)
for i in range(N):
    T[i]=i*dt
p = 1/6 #Connectivity rate High connectivity is just mean field so it looks a lot smoother... want to maybe not make it so high

M=getRandomConnectivity(n,p) #args(n,p)
l=matrixOfEdges(M,n)
M_prime = copy.deepcopy(M)
l_prime = copy.deepcopy(l)
#curr_n = n #Current number of neurons in network 
to_cull = 0

tauv=0.01 #0.01
tauc=0.5 #0.5
Vstar=15 #Constant (according to experiments) 15
Cstar=20 #15
V0=18 #18
C0=18 #18

delV= 4. #1.2
delVmin = 0#delV/3
delC= 0.015 #0.015
r= 70 #70

params=zeros(13)
params[0]=N
params[1]=n
params[2]=dt
params[3]=tauv
params[4]=tauc
params[5]=Vstar
params[6]=Cstar
params[7]=delV
params[8]=delC
params[9]=r
params[10]=V0
params[11]=C0
params[12]=delVmin #connenctivity rate from above.


phasen = []
phaseV = []
kCore = []
kCore_predict = []

dV = 0.25
delVmax = 20.

while True:
    C,V,f,P,Voltages = simdyn3(M_prime,l_prime,params)
    check = classify(V,params)
    while True:
        #import pdb; pdb.set_trace()
        params[1] -= 1
        #curr_n = curr_n-1
        to_cull += 1
        M_prime = killNeuron2(to_cull, M)
        l_prime = matrixOfEdges(M_prime, params[1])
        if params[1] <= 1:
            params[7] += dV
            params[12] = params[7]/3
            params[1] = n
            to_cull = 0
            M_prime = copy.deepcopy(M)
            l_prime = copy.deepcopy(l)
            #curr_n = n
            break
        C,V,f,P,Voltages = simdyn3(M_prime,l_prime,params)
        check2 = classify(V,params)
        if check2 != check:
            print("delV: ", params[7])
            print("transition from", check, "to", check2)
            #print("Kcore: ", largestKcore(M_prime), "Predicted:",floor(Vstar/(tauv*70*params[12])))
            print("Kcore: ", largestKcore(M_prime), "Predicted:",Vstar/(tauv*70*params[12]))
            phasen.append(params[1])
            phaseV.append(params[7])
            kCore.append(largestKcore(M_prime))
            kCore_predict.append(floor(Vstar/(tauv * 70 * params[12])))
            params[7] += dV
            params[12] = params[7]/3
            if params[1] < n-3:
                to_cull -= 3
                params[1] += 3 #Changed from params[1] = n to track boundary 
                M_prime = killNeuron2(to_cull, M)
                l_prime = matrixOfEdges(M_prime, params[1])
            else:
                to_cull = 0
                params[1] = n 
                M_prime = copy.deepcopy(M)
                l_prime = copy.deepcopy(l)
                
            break

    if params[7] > delVmax:
        break
print('loop complete')


delV:  4.25
transition from 3 to 2
Kcore:  15.0 Predicted: 15.126050420168065
delV:  4.5
transition from 3 to 2
Kcore:  14.0 Predicted: 14.285714285714285
delV:  4.75
transition from 3 to 2
Kcore:  13.0 Predicted: 13.533834586466165
delV:  5.0
transition from 3 to 2
Kcore:  12.0 Predicted: 12.857142857142856
delV:  5.25
transition from 3 to 2
Kcore:  12.0 Predicted: 12.244897959183673
delV:  5.5
transition from 3 to 2
Kcore:  11.0 Predicted: 11.688311688311687
delV:  5.75
transition from 3 to 2
Kcore:  11.0 Predicted: 11.180124223602483
delV:  6.0
transition from 3 to 2
Kcore:  10.0 Predicted: 10.714285714285714
delV:  6.25
transition from 3 to 2
Kcore:  10.0 Predicted: 10.285714285714285
delV:  6.5
transition from 3 to 2
Kcore:  9.0 Predicted: 9.89010989010989
delV:  6.75
transition from 3 to 2
Kcore:  9.0 Predicted: 9.523809523809522
delV:  7.0
transition from 3 to 2
Kcore:  9.0 Predicted: 9.183673469387754
delV:  7.25
transition from 3 to 2
Kcore:  8.0 Predicted: 8.866995073891626
d

In [64]:
plot(phaseV, kCore, label="k-core size ")
plot(phaseV, kCore_predict, '--', label="Predicted k-core size") 
legend()
title("k-core vs. $\Delta V$")
xlabel("$\Delta V$")
ylabel("k-core size")
#savefig("figures/kcore_exact_vs_predict.png")

Text(0,0.5,'k-core size')

In [15]:
plt.plot(phaseV, phasen)

In [61]:
plt.plot(C,V)

In [9]:
#Looking at k-core size versus the actual number of firing neurons in the network 
N=10000
n=130 #150
dt=0.001
T=zeros(N)
for i in range(N):
    T[i]=i*dt
p = 1 #Connectivity rate High connectivity is just mean field so it looks a lot smoother... want to maybe not make it so high

M=getRandomConnectivity(n,p) #args(n,p)
M_T = transpose(M)
l1=matrixOfEdges(M,n)
l2=matrixOfEdges(M_T,n)
#M_prime = copy.deepcopy(M_T)
#l_prime = copy.deepcopy(l)

tauv=0.01 #0.01
tauc=0.5 #0.5
Vstar=15 #Constant (according to experiments) 15
Cstar=20 #15
V0=30 #18
C0=30 #18

delV= 1.2 #1.2
delVmin = delV/3
delC= 0.015 #0.015
r= 70 #70

#M_prime = killNeuron2(n-cull_to, M_T)
#l_prime = matrixOfEdges(M_prime, cull_to)

params=zeros(13)
params[0]=N
params[1]=n #instead of actual n, cull neurons til desired value
params[2]=dt
params[3]=tauv
params[4]=tauc
params[5]=Vstar
params[6]=Cstar
params[7]=delV
params[8]=delC
params[9]=r
params[10]=V0
params[11]=C0
params[12]=delVmin #connenctivity rate from above.


#C,V,neuron_counts = simdyn3(M_prime,l_prime,params)
C,V,neuron_counts, P, Voltages = simdyn3(M,l1,params)


#C,V,neuron_counts = simdyn(M, l1, params)
itself = kcoreitself(M,ceil(Vstar/(tauv*delVmin*r)))
plt.plot(C,V)

#plt.plot(C,V)

In [10]:
out = Vstar/(tauv*delVmin*r)

print("Classification:",classify(V,params))
print("Largest kcore:", largestKcore(M))
print("Predicted k: ",out)
print("Shape of kcore (gives kcore size):",itself.shape)
print("Number of neurons total in network:",params[1])
print("Number of firing neurons:",sum(neuron_counts[N-1]))

Classification: 3
Largest kcore: 129.0
Predicted k:  53.57142857142857
Shape of kcore (gives kcore size): (130, 130)
Number of neurons total in network: 130.0
Number of firing neurons: 130.0


In [11]:
C,V,firing_neurons = simdyn(M, l1, params)
plt.plot(C,V)

In [161]:
G = nx.DiGraph()
H = nx.DiGraph()
G.add_nodes_from(range(0,n))
H.add_nodes_from(range(0,n))
for i in range(n):
    for j in range(n):
        if M[i,j] == 1: #and neuron_counts[N-1,j] == 1:
            #if neuron_counts[N-1,j] == 1:
            if neuron_counts[N-1,i] == 1:
                H.add_edge(i,j,firing=True)
            else:
                G.add_edge(i,j,firing=False)

In [162]:
plt.title("%d Neurons in network"%G.order())
nx.draw_circular(H,node_size=20,width=0.3,node_color='black', edge_color='red',with_labels=True)
nx.draw_circular(G,node_size=20,width=0.3,edge_color='black',node_color='black')
plt.savefig('neuron_connectivity.png')

### Mean Field Analysis with Step Function 

In [38]:
#Step Function Toy Model 

#similar to simdyn from before but for mean field case 
@jit(nopython=True)
def mean_field_dynamics(params):
    N = int(params[0])
    n = int(params[1]) 
    dt = params[2]
    tauv = params[3]
    tauc = params[4] 
    Vstar = params[5] 
    Cstar = params[6]
    delV = params[7]
    delC = params[8]
    r_1 = params[9]
    r_0 = params[10]
    V_0 = params[11]
    C_0 = params[12]
    
    V=zeros(N*n).reshape(N,n)  #now we have potencial and calcium concentration for each neuron
    C=zeros(N*n).reshape(N,n)
    
    firing_neurons_count = zeros(N*n).reshape(N,n) #Individually count how may neurons are actually firing in this model at a given step N.
    
    avV=zeros(N) #average potencial
    avC=zeros(N)  #average calcium
    
    for i in range(n):
        V[0,i]=V_0 #initial conditions
        C[0,i]=C_0
        if V[0,i] > Vstar: #keep track of and return how many neurons in the network are actually firing  
            firing_neurons_count[0,i] = 1
        avV[0]=n*V_0  #we will divide to n later
        avC[0]=n*C_0
    
    for i in range(N - 1): 
        for j in range(n):
            if V[i,j] > Vstar: 
                r = r_1
            else:
                r = 0

            if C[i,j] < Cstar: 
                dV = delV 
            else: 
                dV = 0

            V[i+1,j] = (-V[i,j]/tauv + n*dV*(r + r_0)) * dt + V[i,j]
            if V[i+1,j] > Vstar: 
                firing_neurons_count[i+1,j] = 1
            C[i+1,j] = (-C[i,j]/tauc + n*delC*(r + r_0)) * dt + C[i,j]
            avC[i+1]=avC[i+1]+C[i+1,j]
            avV[i+1]=avV[i+1]+V[i+1,j]
            
    for i in range (N):
        avC[i]=avC[i]/n
        avV[i]=avV[i]/n    
    
    return avC, avV, firing_neurons_count 
    
    

N = 10000 #Number of Steps 
dt = 0.001

T = zeros(N)
#V = zeros(N) 
#C = zeros(N)

#V[0] = 18
#C[0] = 18

V_0, C_0 = 18, 18

tauv = 0.01
tauc = 0.5

Vstar = 15#15
Cstar = 20#20

n = 100 #100

delV = 1. #1.
delC = 0.015#0.015
r_1 = 70 #3000
r_0 = 5 #10

conditions = tauc*delC*r_1 > Cstar and tauc*delC*r_0 < Cstar and delV*tauv*r_0 > Vstar

params = zeros(13)
params[0] = N
params[1] = n
params[2] = dt 
params[3] = tauv 
params[4] = tauc 
params[5] = Vstar
params[6] = Cstar 
params[7] = delV 
params[8] = delC 
params[9] = r_1 
params[10] = r_0 
params[11] = V_0
params[12] = C_0

C,V,f = mean_field_dynamics(params)

plt.plot(C,V)
print("Classification:", classify(V,params))
print("Conditions for SO phase met:", conditions)

plt.title("Stable Oscillations")
plt.xlabel("C")
plt.ylabel("V")
#plt.savefig("figures/step_SO.png")

        

Classification: 2
Conditions for SO phase met: False


Text(0,0.5,'V')

In [27]:
#Phase diagram in delV 
#phasen, phaseV, phasen_SO, phasen_Q, phase_SO, phase_Q = [],[],[],[],[],[]
phasen, phaseV = [],[] #for plotting general, uninformative phase diagram
phasen_SO_Q, phasen_Q_HA, phasen_SO_HA, phase_SO_Q, phase_Q_HA, phase_SO_HA = [],[],[],[],[],[]
dV = 0.01
delVmax = 10.

#For Q phase 
while True:
    C,V,f = mean_field_dynamics(params)
    check = classify(V,params)
    while True:
        params[1] -= 1
        if params[1] <= 1:
            params[7] += dV
            params[1] = n
            break
        C,V,f = mean_field_dynamics(params)
        check2 = classify(V,params)
        if check2 != check:
            print("delV: ", params[7])
            print("transition from", check, "to", check2)
            phasen.append(params[1])
            phaseV.append(params[7])
            #Added to plot the two different phase boundaries 
            '''
            if check == 2: 
                phase_Q.append(params[7])
                phasen_Q.append(params[1])
            elif check == 1: 
                phase_SO.append(params[7])
                phasen_SO.append(params[1])
            '''
            if check == 1 and check2 == 2: 
                phasen_SO_Q.append(params[1])
                phase_SO_Q.append(params[7])
            elif check == 1 and check2 == 3:
                phasen_SO_HA.append(params[1])
                phase_SO_HA.append(params[7])
            elif check == 2 and check2 == 3: 
                phasen_Q_HA.append(params[1])
                phase_Q_HA.append(params[7])
            #End Inserted Code
            #params[7] += dV
            #params[1] = n
            break
    if params[7] > delVmax:
        break
        
print('loop complete') 

delV:  1.0
transition from 2 to 3
delV:  1.0
transition from 3 to 2
delV:  1.01
transition from 2 to 3
delV:  1.01
transition from 3 to 2
delV:  1.02
transition from 2 to 3
delV:  1.02
transition from 3 to 2
delV:  1.03
transition from 2 to 3
delV:  1.03
transition from 3 to 2
delV:  1.04
transition from 2 to 3
delV:  1.04
transition from 3 to 2
delV:  1.05
transition from 2 to 3
delV:  1.05
transition from 3 to 2
delV:  1.06
transition from 2 to 3
delV:  1.06
transition from 3 to 2
delV:  1.07
transition from 2 to 3
delV:  1.07
transition from 3 to 2
delV:  1.08
transition from 2 to 3
delV:  1.08
transition from 3 to 2
delV:  1.09
transition from 2 to 3
delV:  1.09
transition from 3 to 2
delV:  1.1
transition from 2 to 3
delV:  1.1
transition from 3 to 2
delV:  1.11
transition from 2 to 3
delV:  1.11
transition from 3 to 2
delV:  1.12
transition from 2 to 3
delV:  1.12
transition from 3 to 2
delV:  1.1300000000000001
transition from 2 to 3
delV:  1.1300000000000001
transition from 3 t

delV:  1.8800000000000008
transition from 2 to 3
delV:  1.8800000000000008
transition from 3 to 2
delV:  1.8900000000000008
transition from 2 to 3
delV:  1.8900000000000008
transition from 3 to 2
delV:  1.9000000000000008
transition from 2 to 3
delV:  1.9000000000000008
transition from 3 to 2
delV:  1.9100000000000008
transition from 2 to 3
delV:  1.9100000000000008
transition from 3 to 2
delV:  1.9200000000000008
transition from 2 to 3
delV:  1.9200000000000008
transition from 3 to 2
delV:  1.9300000000000008
transition from 2 to 3
delV:  1.9300000000000008
transition from 3 to 2
delV:  1.9400000000000008
transition from 2 to 3
delV:  1.9400000000000008
transition from 3 to 2
delV:  1.9500000000000008
transition from 2 to 3
delV:  1.9500000000000008
transition from 3 to 2
delV:  1.9600000000000009
transition from 2 to 3
delV:  1.9600000000000009
transition from 3 to 2
delV:  1.9700000000000009
transition from 2 to 3
delV:  1.9700000000000009
transition from 3 to 2
delV:  1.98000000000

delV:  2.7299999999999853
transition from 2 to 3
delV:  2.7299999999999853
transition from 3 to 2
delV:  2.739999999999985
transition from 2 to 3
delV:  2.739999999999985
transition from 3 to 2
delV:  2.749999999999985
transition from 2 to 3
delV:  2.749999999999985
transition from 3 to 2
delV:  2.7599999999999847
transition from 2 to 3
delV:  2.7599999999999847
transition from 3 to 2
delV:  2.7699999999999845
transition from 2 to 3
delV:  2.7699999999999845
transition from 3 to 2
delV:  2.7799999999999843
transition from 2 to 3
delV:  2.7799999999999843
transition from 3 to 2
delV:  2.789999999999984
transition from 2 to 3
delV:  2.789999999999984
transition from 3 to 2
delV:  2.799999999999984
transition from 2 to 3
delV:  2.799999999999984
transition from 3 to 2
delV:  2.8099999999999836
transition from 2 to 3
delV:  2.8099999999999836
transition from 3 to 2
delV:  2.8199999999999834
transition from 2 to 3
delV:  2.8199999999999834
transition from 3 to 2
delV:  2.829999999999983
tra

delV:  3.3899999999999713
transition from 2 to 3
delV:  3.3899999999999713
transition from 3 to 2
delV:  3.399999999999971
transition from 1 to 2
delV:  3.399999999999971
transition from 2 to 3
delV:  3.399999999999971
transition from 3 to 2
delV:  3.409999999999971
transition from 1 to 2
delV:  3.409999999999971
transition from 2 to 3
delV:  3.409999999999971
transition from 3 to 2
delV:  3.4199999999999706
transition from 1 to 2
delV:  3.4199999999999706
transition from 2 to 3
delV:  3.4199999999999706
transition from 3 to 2
delV:  3.4299999999999704
transition from 1 to 2
delV:  3.4299999999999704
transition from 2 to 3
delV:  3.4299999999999704
transition from 3 to 2
delV:  3.43999999999997
transition from 1 to 2
delV:  3.43999999999997
transition from 2 to 3
delV:  3.43999999999997
transition from 3 to 2
delV:  3.44999999999997
transition from 1 to 2
delV:  3.44999999999997
transition from 2 to 3
delV:  3.44999999999997
transition from 3 to 2
delV:  3.4599999999999698
transition f

delV:  3.959999999999959
transition from 1 to 2
delV:  3.959999999999959
transition from 2 to 3
delV:  3.959999999999959
transition from 3 to 2
delV:  3.969999999999959
transition from 1 to 2
delV:  3.969999999999959
transition from 2 to 3
delV:  3.969999999999959
transition from 3 to 2
delV:  3.9799999999999587
transition from 1 to 2
delV:  3.9799999999999587
transition from 2 to 3
delV:  3.9799999999999587
transition from 3 to 2
delV:  3.9899999999999585
transition from 1 to 2
delV:  3.9899999999999585
transition from 2 to 3
delV:  3.9899999999999585
transition from 3 to 2
delV:  3.9999999999999583
transition from 1 to 2
delV:  3.9999999999999583
transition from 2 to 3
delV:  3.9999999999999583
transition from 3 to 2
delV:  4.009999999999958
transition from 1 to 2
delV:  4.009999999999958
transition from 2 to 3
delV:  4.009999999999958
transition from 3 to 2
delV:  4.019999999999958
transition from 1 to 2
delV:  4.019999999999958
transition from 2 to 3
delV:  4.019999999999958
transi

...

delV:  6.809999999999898
transition from 1 to 2
delV:  6.809999999999898
transition from 2 to 3
delV:  6.809999999999898
transition from 3 to 2
delV:  6.819999999999898
transition from 1 to 2
delV:  6.819999999999898
transition from 2 to 3
delV:  6.819999999999898
transition from 3 to 2
delV:  6.829999999999898
transition from 1 to 2
delV:  6.829999999999898
transition from 2 to 3
delV:  6.829999999999898
transition from 3 to 2
delV:  6.839999999999898
transition from 1 to 2
delV:  6.839999999999898
transition from 2 to 3
delV:  6.839999999999898
transition from 3 to 2
delV:  6.8499999999998975
transition from 1 to 2
delV:  6.8499999999998975
transition from 2 to 3
delV:  6.8499999999998975
transition from 3 to 2
delV:  6.859999999999897
transition from 1 to 2
delV:  6.859999999999897
transition from 2 to 3
delV:  6.859999999999897
transition from 3 to 2
delV:  6.869999999999897
transition from 1 to 2
delV:  6.869999999999897
transition from 2 to 3
delV:  6.869999999999897
transition f

delV:  7.379999999999886
transition from 1 to 2
delV:  7.379999999999886
transition from 2 to 3
delV:  7.379999999999886
transition from 3 to 2
delV:  7.389999999999886
transition from 1 to 2
delV:  7.389999999999886
transition from 2 to 3
delV:  7.389999999999886
transition from 3 to 2
delV:  7.399999999999886
transition from 1 to 2
delV:  7.399999999999886
transition from 2 to 3
delV:  7.399999999999886
transition from 3 to 2
delV:  7.409999999999886
transition from 1 to 2
delV:  7.409999999999886
transition from 2 to 3
delV:  7.409999999999886
transition from 3 to 2
delV:  7.419999999999885
transition from 1 to 2
delV:  7.419999999999885
transition from 2 to 3
delV:  7.419999999999885
transition from 3 to 2
delV:  7.429999999999885
transition from 1 to 2
delV:  7.429999999999885
transition from 2 to 3
delV:  7.429999999999885
transition from 3 to 2
delV:  7.439999999999885
transition from 1 to 2
delV:  7.439999999999885
transition from 2 to 3
delV:  7.439999999999885
transition from

delV:  7.949999999999874
transition from 1 to 2
delV:  7.949999999999874
transition from 2 to 3
delV:  7.949999999999874
transition from 3 to 2
delV:  7.959999999999874
transition from 1 to 2
delV:  7.959999999999874
transition from 2 to 3
delV:  7.959999999999874
transition from 3 to 2
delV:  7.969999999999874
transition from 1 to 2
delV:  7.969999999999874
transition from 2 to 3
delV:  7.969999999999874
transition from 3 to 2
delV:  7.979999999999873
transition from 1 to 2
delV:  7.979999999999873
transition from 2 to 3
delV:  7.979999999999873
transition from 3 to 2
delV:  7.989999999999873
transition from 1 to 2
delV:  7.989999999999873
transition from 2 to 3
delV:  7.989999999999873
transition from 3 to 2
delV:  7.999999999999873
transition from 1 to 2
delV:  7.999999999999873
transition from 2 to 3
delV:  7.999999999999873
transition from 3 to 2
delV:  8.009999999999874
transition from 1 to 2
delV:  8.009999999999874
transition from 2 to 3
delV:  8.009999999999874
transition from

delV:  8.61999999999986
transition from 1 to 3
delV:  8.61999999999986
transition from 3 to 2
delV:  8.62999999999986
transition from 1 to 3
delV:  8.62999999999986
transition from 3 to 2
delV:  8.63999999999986
transition from 1 to 3
delV:  8.63999999999986
transition from 3 to 2
delV:  8.64999999999986
transition from 1 to 3
delV:  8.64999999999986
transition from 3 to 2
delV:  8.65999999999986
transition from 1 to 3
delV:  8.65999999999986
transition from 3 to 2
delV:  8.66999999999986
transition from 1 to 3
delV:  8.66999999999986
transition from 3 to 2
delV:  8.67999999999986
transition from 1 to 3
delV:  8.67999999999986
transition from 3 to 2
delV:  8.68999999999986
transition from 1 to 3
delV:  8.68999999999986
transition from 3 to 2
delV:  8.699999999999859
transition from 1 to 3
delV:  8.699999999999859
transition from 3 to 2
delV:  8.709999999999859
transition from 1 to 3
delV:  8.709999999999859
transition from 3 to 2
delV:  8.719999999999859
transition from 1 to 3
delV:  8

delV:  9.479999999999842
transition from 1 to 3
delV:  9.479999999999842
transition from 3 to 2
delV:  9.489999999999842
transition from 1 to 3
delV:  9.489999999999842
transition from 3 to 2
delV:  9.499999999999842
transition from 1 to 3
delV:  9.499999999999842
transition from 3 to 2
delV:  9.509999999999842
transition from 1 to 3
delV:  9.509999999999842
transition from 3 to 2
delV:  9.519999999999841
transition from 1 to 3
delV:  9.519999999999841
transition from 3 to 2
delV:  9.529999999999841
transition from 1 to 3
delV:  9.529999999999841
transition from 3 to 2
delV:  9.539999999999841
transition from 1 to 3
delV:  9.539999999999841
transition from 3 to 2
delV:  9.54999999999984
transition from 1 to 3
delV:  9.54999999999984
transition from 3 to 2
delV:  9.55999999999984
transition from 1 to 3
delV:  9.55999999999984
transition from 3 to 2
delV:  9.56999999999984
transition from 1 to 3
delV:  9.56999999999984
transition from 3 to 2
delV:  9.57999999999984
transition from 1 to 3

In [31]:
#Plotting
plot(phase_SO_Q, phasen_SO_Q, label="SO-Q boundary", color="red")
plot(phase_SO_HA, phasen_SO_HA, label="SO-HA boundary", color="blue")
plot(phase_Q_HA, phasen_Q_HA, label="Q-HA boundary", color="green")
plt.legend()
plt.xlabel("Voltage ($\Delta$V)")
plt.ylabel("Neuron Count (n)")
plt.title("Neuron Count vs. Voltage Phase Diagram. $\Delta$C = %f"%delC)
plt.text(6, 37.2, "Q")
plt.text(9., 37.2, "SO")
plt.text(8,32, "HA")
plt.savefig("figures/step_nV_phase.png")

In [45]:
#Phase diagram in delC 

delC = 0.
delV = 5.
#phasen, phaseC, phasen_SO, phasen_Q, phasen_HA, phase_SO, phase_Q, phase_HA = [],[],[],[],[],[],[],[]
phasen, phaseC = [],[] #for plotting general, uninformative phase diagram
phasen_SO_Q, phasen_Q_HA, phasen_SO_HA, phase_SO_Q, phase_Q_HA, phase_SO_HA = [],[],[],[],[],[]

#Just to reset all parameters 
params = zeros(13)
params[0] = N
params[1] = n
params[2] = dt 
params[3] = tauv 
params[4] = tauc 
params[5] = Vstar
params[6] = Cstar 
params[7] = delV 
params[8] = delC 
params[9] = r_1 
params[10] = r_0 
params[11] = V_0
params[12] = C_0

#phasen, phaseC = [],[]
dC = 0.0002 #0.001
delCmax = .04

while True:
    C,V,f = mean_field_dynamics(params)
    check = classify(V,params)
    while True:
        params[1] -= 1
        if params[1] <= 1:
            params[8] += dC
            params[1] = n
            break
        C,V,f = mean_field_dynamics(params)
        check2 = classify(V,params)
        if check2 != check:
            print("delC: ", params[8])
            print("transition from", check, "to", check2)
            phasen.append(params[1])
            phaseC.append(params[8])
            '''
            if check == 2: 
                phase_Q.append(params[8])
                phasen_Q.append(params[1])
            elif check == 1: 
                phase_SO.append(params[8])
                phasen_SO.append(params[1])
            else:
                phase_HA.append(params[8])
                phasen_HA.append(params[1])
            '''
            if check == 1 and check2 == 2: 
                phasen_SO_Q.append(params[1])
                phase_SO_Q.append(params[8])
            elif check == 1 and check2 == 3:
                phasen_SO_HA.append(params[1])
                phase_SO_HA.append(params[8])
            elif check == 2 and check2 == 3: 
                phasen_Q_HA.append(params[1])
                phase_Q_HA.append(params[8])
            #params[8] += dC
            #if params[1] < n-3:
            #    params[1] += 3 #Changed from params[1] = n to track boundary 
            #else:
            #    params[1] = n 
            #params[1] = n
            break
    if params[8] > delCmax:
        break
print("loop complete")

delC:  0.0
transition from 3 to 2
delC:  0.0002
transition from 3 to 2
delC:  0.0004
transition from 3 to 2
delC:  0.0006000000000000001
transition from 3 to 2
delC:  0.0008
transition from 3 to 2
delC:  0.001
transition from 3 to 2
delC:  0.0012000000000000001
transition from 3 to 2
delC:  0.0014000000000000002
transition from 3 to 2
delC:  0.0016000000000000003
transition from 3 to 2
delC:  0.0018000000000000004
transition from 3 to 2
delC:  0.0020000000000000005
transition from 3 to 2
delC:  0.0022000000000000006
transition from 3 to 2
delC:  0.0024000000000000007
transition from 3 to 2
delC:  0.0026000000000000007
transition from 3 to 2
delC:  0.002800000000000001
transition from 3 to 2
delC:  0.003000000000000001
transition from 3 to 2
delC:  0.003200000000000001
transition from 3 to 2
delC:  0.003400000000000001
transition from 3 to 2
delC:  0.003600000000000001
transition from 3 to 2
delC:  0.0038000000000000013
transition from 3 to 2
delC:  0.004000000000000001
transition from 

delC:  0.015800000000000015
transition from 3 to 2
delC:  0.016000000000000014
transition from 1 to 2
delC:  0.016000000000000014
transition from 2 to 3
delC:  0.016000000000000014
transition from 3 to 2
delC:  0.016200000000000013
transition from 1 to 2
delC:  0.016200000000000013
transition from 2 to 3
delC:  0.016200000000000013
transition from 3 to 2
delC:  0.016400000000000012
transition from 1 to 2
delC:  0.016400000000000012
transition from 2 to 3
delC:  0.016400000000000012
transition from 3 to 2
delC:  0.01660000000000001
transition from 1 to 2
delC:  0.01660000000000001
transition from 2 to 3
delC:  0.01660000000000001
transition from 3 to 2
delC:  0.01680000000000001
transition from 1 to 2
delC:  0.01680000000000001
transition from 2 to 3
delC:  0.01680000000000001
transition from 3 to 2
delC:  0.017000000000000008
transition from 1 to 2
delC:  0.017000000000000008
transition from 2 to 3
delC:  0.017000000000000008
transition from 3 to 2
delC:  0.017200000000000007
transitio

delC:  0.026999999999999948
transition from 1 to 2
delC:  0.026999999999999948
transition from 2 to 3
delC:  0.026999999999999948
transition from 3 to 2
delC:  0.027199999999999946
transition from 1 to 2
delC:  0.027199999999999946
transition from 2 to 3
delC:  0.027199999999999946
transition from 3 to 2
delC:  0.027399999999999945
transition from 1 to 2
delC:  0.027399999999999945
transition from 2 to 3
delC:  0.027399999999999945
transition from 3 to 2
delC:  0.027599999999999944
transition from 1 to 2
delC:  0.027599999999999944
transition from 2 to 3
delC:  0.027599999999999944
transition from 3 to 2
delC:  0.027799999999999943
transition from 1 to 2
delC:  0.027799999999999943
transition from 2 to 3
delC:  0.027799999999999943
transition from 3 to 2
delC:  0.02799999999999994
transition from 1 to 2
delC:  0.02799999999999994
transition from 2 to 3
delC:  0.02799999999999994
transition from 3 to 2
delC:  0.02819999999999994
transition from 1 to 2
delC:  0.02819999999999994
transiti

delC:  0.03779999999999988
transition from 3 to 2
delC:  0.03799999999999988
transition from 1 to 2
delC:  0.03799999999999988
transition from 2 to 3
delC:  0.03799999999999988
transition from 3 to 2
delC:  0.03819999999999988
transition from 1 to 2
delC:  0.03819999999999988
transition from 2 to 3
delC:  0.03819999999999988
transition from 3 to 2
delC:  0.03839999999999988
transition from 1 to 2
delC:  0.03839999999999988
transition from 2 to 3
delC:  0.03839999999999988
transition from 3 to 2
delC:  0.03859999999999988
transition from 1 to 2
delC:  0.03859999999999988
transition from 2 to 3
delC:  0.03859999999999988
transition from 3 to 2
delC:  0.038799999999999876
transition from 1 to 2
delC:  0.038799999999999876
transition from 2 to 3
delC:  0.038799999999999876
transition from 3 to 2
delC:  0.038999999999999875
transition from 1 to 2
delC:  0.038999999999999875
transition from 2 to 3
delC:  0.038999999999999875
transition from 3 to 2
delC:  0.039199999999999874
transition from 

In [47]:
plot(phase_SO_Q, phasen_SO_Q, label="SO-Q boundary", color="red")
plot(phase_SO_HA, phasen_SO_HA, label="SO-HA boundary", color="blue")
plot(phase_Q_HA, phasen_Q_HA, label="Q-HA boundary", color="green")
legend()
plt.xlabel("Calcium ($\Delta$C)")
plt.ylabel("Neuron Count (n)")
plt.title("Neuron Count vs. Calcium Phase Diagram. $\Delta$V = %f"%(delV))
plt.text(.008, 45,"HA")
plt.text(.015, 50, "Q")
plt.text(.015, 65, "SO")
plt.savefig("figures/step_nC_phase.png")

In [22]:
#Plotting 
plot(phase_Q, phasen_Q, label="Quiescent boundary", color="red")
plot(phase_SO, phasen_SO, label="SO boundary", color="blue")
plot(phase_HA, phasen_HA, label="HA boundary", color="green")
plt.legend()
plt.xlabel("Calcium ($\Delta$C)")
plt.ylabel("Neuron Count (n)")
plt.title("Neuron Count vs. Calcium Phase Diagram. $\Delta$V = %f"%(delV))
plt.text(.008, 45,"HA")
plt.text(.015, 50, "Q")
plt.text(.015, 65, "SO")
plt.text(.0125, 1, "Q")
#plt.savefig("figures/step_nC_phase.png")

Text(0.0125,1,'Q')

### Here I plot phase diagrams for the Step Function using a randomly connected matrix 

In [8]:
def simdyn4(M, l, params):
    return simDynamics4(M, l, params)

@jit(nopython=True)
def simDynamics4(M, l, params):  #params is a vector of parameters
    N=int(params[0])
    n=int(params[1])
    dt=params[2]
    tauv=params[3]
    tauc=params[4]
    Vstar=params[5]
    Cstar=params[6]
    delV=params[7]
    delC=params[8]
    r_1=params[9]
    V0=params[10]
    C0=params[11]
    delVmin=params[12]
    r_0=params[13]
    V=zeros(N*n).reshape(N,n)  #now we have potencial and calcium concentration for each neuron
    C=zeros(N*n).reshape(N,n)
    #P=zeros(n)  # function P(V), sigmoid
    P = zeros(N*n).reshape(N,n)
    deltaV=zeros(n)
    sump=zeros(n)  #sum of P over the vertices
    avV=zeros(N) #average potencial
    avC=zeros(N)  #average calcium
    firing_neurons_count = zeros(N*n).reshape(N,n) #Individually count how may neurons are actually firing in this model at a given step N.
    
    for i in range(n):
        V[0,i]=V0 #initial conditions
        C[0,i]=C0 
        if V[0,i] > Vstar: #keep track of and return how many neurons in the network are actually firing  
            firing_neurons_count[0,i] = 1
        avV[0]=n*V0  #we will divide to n later
        avC[0]=n*C0
    for i in range (N-1):
        for j in range(n):
            if (V[i,j]>Vstar): #V[i,j] stores voltage of each neuron instead of returning the average voltage 
                P[i,j]=r_1+r_0
            else:
                P[i,j]=r_0
            if (C[i,j]<Cstar):
                deltaV[j]=delV #change from 1.2 
            else:
                deltaV[j]=delVmin #changed from .4
        for j in range(n):
            a=int(l[j,0]) #a should give number of firing neighbors 
            for k in range(a):
                #b=int(l[j,k])
                b = int(l[j,k+1])
                sump[j]=sump[j]+P[i,b]  #if neuron connected to j-th neuron it participates in sum
        for j in range(n):
            V[i+1,j]=(-V[i,j]/tauv + deltaV[j]*sump[j])*dt + V[i,j]
            if V[i+1,j] > Vstar: 
                firing_neurons_count[i+1,j] = 1 #used to return count of firing neurons later
            #if ((firing_neurons_count[i,j] == 1) and (firing_neurons_count[i+1,j] == 0)):
            #    print(sump[j])
            #C[i+1,j]=(-C[i,j]/tauc + delC*(r_1+r_0)*sump[j])*dt + C[i,j]
            C[i+1,j]=(-C[i,j]/tauc + delC*sump[j])*dt + C[i,j]
            avC[i+1]=avC[i+1]+C[i+1,j]
            avV[i+1]=avV[i+1]+V[i+1,j]
        for j in range(n):
            sump[j]=0
    for i in range (N):
        avC[i]=avC[i]/n
        avV[i]=avV[i]/n
        
        #plot([avC[i],avC[i+1]],[avV[i],avV[i+1]])    
    return(avC,avV, firing_neurons_count, P, V)

In [15]:
#Network behavior for one set of parameters 
N=10000
n=100 #150
dt=0.001
T=zeros(N)
for i in range(N):
    T[i]=i*dt
p = .99 #1/6 in real world 

M=getRandomConnectivity(n,p) #args(n,p)
l=matrixOfEdges(M,n)
M_prime = copy.deepcopy(M)
l_prime = copy.deepcopy(l)
to_cull = 0

tauv=0.01 #0.01
tauc=0.5 #0.5
Vstar=15 #Constant (according to experiments) 15
Cstar=20 #15
V0=18 #18
C0=18 #18

delV= 10. #1.2
delVmin = 0 #delV/3
delC= 0.015 #0.015
r_1= 70 #Max Firing rate is r_1 + r_0 
r_0 = 5 #Min firing rate 

#Instantiate array of parameters 
params=zeros(14)
params[0]=N
params[1]=n #instead of actual n, cull neurons til desired value
params[2]=dt
params[3]=tauv
params[4]=tauc
params[5]=Vstar
params[6]=Cstar
params[7]=delV
params[8]=delC
params[9]=r_1
params[10]=V0
params[11]=C0
params[12]=delVmin #connenctivity rate from above.
params[13]=r_0 

C,V,f,P,Voltages = simdyn4(M,l, params) 
plot(C,V)
ylabel("V")
xlabel("C")
title("Randomly Connected Network")
#savefig("step_rand.png") 
print(classify(V,params))
print(M)

1
[[0. 1. 1. ... 1. 0. 1.]
 [1. 0. 1. ... 1. 1. 1.]
 [1. 1. 0. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 1. 0. 0.]
 [1. 1. 1. ... 1. 1. 0.]]


In [10]:
#Phase diagram in delV 
#phasen, phaseV, phasen_SO, phasen_Q, phase_SO, phase_Q = [],[],[],[],[],[]
phasen, phaseV, kCore = [],[],[] #for plotting general, uninformative phase diagram
#phasen_SO_Q, phasen_Q_HA, phasen_SO_HA, phase_SO_Q, phase_Q_HA, phase_SO_HA = [],[],[],[],[],[]
dV = 0.1
delVmax = 10.
to_cull = 0

while True:
    C,V,f,P,Voltages = simdyn4(M_prime,l_prime,params)
    check = classify(V,params)
    while True:
        params[1] -= 1
        to_cull += 1
        M_prime = killNeuron2(to_cull, M)
        l_prime = matrixOfEdges(M_prime, params[1])
        if params[1] <= 1:
            params[7] += dV
            params[12] = params[7]/3
            params[1] = n
            to_cull = 0
            M_prime = copy.deepcopy(M)
            l_prime = copy.deepcopy(l)
            break
        C,V,f,P,Voltages = simdyn4(M_prime,l_prime,params)
        check2 = classify(V,params)
        if check2 != check:
            print("delV: ", params[7])
            print("transition from", check, "to", check2)
            print("Kcore: ", largestKcore(M_prime), "Predicted:",Vstar/(tauv*70*params[12]))
            phasen.append(params[1])
            phaseV.append(params[7])
            kCore.append(largestKcore(M_prime))
            params[7] += dV
            params[12] = params[7]/3
            if params[1] < n-3:
                to_cull -= 3
                params[1] += 3 #Changed from params[1] = n to track boundary 
                M_prime = killNeuron2(to_cull, M)
                l_prime = matrixOfEdges(M_prime, params[1])
            else:
                to_cull = 0
                params[1] = n 
                M_prime = copy.deepcopy(M)
                l_prime = copy.deepcopy(l)
                
            break

    if params[7] > delVmax:
        break
print('loop complete')

delV:  0.6
transition from 2 to 3
Kcore:  35.0 Predicted: 107.14285714285714
delV:  0.7
transition from 2 to 3
Kcore:  35.0 Predicted: 91.83673469387756
delV:  0.7999999999999999
transition from 2 to 3
Kcore:  35.0 Predicted: 80.35714285714285
delV:  0.8999999999999999
transition from 2 to 3
Kcore:  35.0 Predicted: 71.42857142857142
delV:  0.9999999999999999
transition from 2 to 3
Kcore:  35.0 Predicted: 64.28571428571428
delV:  1.0999999999999999
transition from 2 to 3
Kcore:  35.0 Predicted: 58.44155844155844
delV:  1.2
transition from 2 to 3
Kcore:  35.0 Predicted: 53.57142857142857
delV:  1.3
transition from 2 to 3
Kcore:  35.0 Predicted: 49.45054945054944
delV:  1.4000000000000001
transition from 2 to 3
Kcore:  35.0 Predicted: 45.918367346938766
delV:  1.5000000000000002
transition from 2 to 3
Kcore:  35.0 Predicted: 42.85714285714285
delV:  1.6000000000000003
transition from 3 to 2
Kcore:  37.0 Predicted: 40.178571428571416
delV:  1.7000000000000004
transition from 3 to 2
Kcore: 

In [11]:
plot(phaseV, phasen)

###  Looking at a highly squished sigmoid function to compare to step function 

In [41]:
#simulation of the situation when all neurons are connected

N=10000   #number of steps
dt=0.001 #time step in milliseconds
#prepare arrays. First we need to set an array of zeros with necessary size and then change its elements

T=zeros(N)  #array of times
V=zeros(N)  #array of potentials as function of time
C=zeros(N)  #array of calcium concentration

#initial potential and calcium concentration
V[0]=18
C[0]=18
#time constants in milliseconds
tauv=0.01
tauc=0.5

#threshold values for potential and calcium concentration
Vstar=15
Cstar=20

#number of neurons
n=100

g_v = .1
g_c = .1

off_set = 1

delV=10 #constant defining how strong neurons firing affect potential
delC=0.015 #constant defining how strong neurons firing affect calcium concentration
r=70  #rate of firing in Hz. Does not present in the equation in the paper because can be hidden in the redefinition of delV and delC

for i in range(N-1):  #numerical solution of the differential equation. This numerical method, called Newton's, is the most trivial and known for not working 
    #in some cases. We will start with it and later can compare with other, more sophisticated methods. You can google and try it yourself. 
    V[i+1]=(-V[i]/tauv + n*delV*r*(1/(1+exp((Vstar-V[i])/g_v)))*(1/(1+exp(C[i]-Cstar)) + off_set) )*dt + V[i]  #step function is defined as sigmoid 1/(1+exp)
    C[i+1]=(-C[i]/tauc + delC*n*r*(1/(1+exp((Vstar-V[i])/g_c))))*dt + C[i]

plot(C,V) #we plot phase diagram concentration vs potential. 


Simple plot of sigmoid to play around with 

In [35]:
x = []
for i in linspace(-20,20,1000):
    x.append(1/(1+exp(-i/g_v)) + off_set) 

plot(x)